In [7]:
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.externals import joblib
from os import walk

import time

import xlrd

%matplotlib inline

In [8]:
#Import File
filename='ZhangData.xlsx'
book=xlrd.open_workbook(filename)

#Extraction
nsheets=book.nsheets

sheet_names=book.sheet_names()

nrows=book.sheet_by_name(sheet_names[0]).nrows
current_header=book.sheet_by_name(sheet_names[0]).row_values(0) 
current_data=[book.sheet_by_name(sheet_names[0]).row_values(i) for i in range(1, nrows)]
df=pd.DataFrame(current_data, columns=current_header)   #DataFrame Construction

sheet_names[0]
'''
for sheet_name in sheet_names:    
    nrows=book.sheet_by_name(sheet_name).nrows
    current_header=book.sheet_by_name(sheet_name).row_values(2) 
    current_data=[book.sheet_by_name(sheet_name).row_values(i) for i in range(3, nrows)]
    sheets[sheet_name]=pd.DataFrame(current_data, columns=current_header)   #DataFrame Construction
#Feedback
print('Data Extracted!')
'''

"\nfor sheet_name in sheet_names:    \n    nrows=book.sheet_by_name(sheet_name).nrows\n    current_header=book.sheet_by_name(sheet_name).row_values(2) \n    current_data=[book.sheet_by_name(sheet_name).row_values(i) for i in range(3, nrows)]\n    sheets[sheet_name]=pd.DataFrame(current_data, columns=current_header)   #DataFrame Construction\n#Feedback\nprint('Data Extracted!')\n"

In [9]:
type(df['ER'])

pandas.core.series.Series

In [10]:
df

,编号,患者,group,年龄,大小,临床分期,ER,PR,ER2,PR2,...,基线含氧量（E0）,含氧量（E1）,Δ氧t1,含氧量（E2）,Δ氧t2,HIF-1,TWIST-1,ITGB,RCB分级,米勒分级
0,4.0,4,1.0,42.0,7.8,3.0,0.0,0.0,0.0,0.0,...,0.306,0.734,1.398693,0.967,2.160131,25.0,25.0,23.0,1.0,5.0
1,26.0,26,1.0,42.0,7.5,3.0,0.0,0.0,0.0,0.0,...,0.145,0.253,0.744828,0.435,2.000000,35.0,30.0,24.0,1.0,5.0
2,45.0,45,1.0,42.0,5.1,3.0,0.0,0.0,0.0,0.0,...,0.322,0.508,0.577640,0.700,1.173913,34.0,29.0,23.0,1.0,5.0
3,47.0,47,1.0,42.0,6.1,3.0,0.0,0.0,0.0,0.0,...,0.504,0.567,0.125000,0.848,0.682540,31.0,26.0,22.0,1.0,5.0
4,89.0,王玉琴,1.0,57.0,4.1,2.0,0.0,0.0,0.0,0.0,...,0.147,0.361,1.455782,0.683,3.646259,35.0,30.0,27.0,1.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,10.0,10,3.0,32.0,2.7,2.0,95.0,95.0,1.0,1.0,...,0.100,0.248,1.480000,0.442,3.420000,79.0,72.0,70.0,3.0,2.0
100,72.0,霍金双,3.0,62.0,2.3,2.0,95.0,95.0,1.0,1.0,...,0.036,0.200,4.555556,0.228,5.333333,89.0,85.0,82.0,3.0,2.0
101,36.0,36,3.0,64.0,5.5,3.0,95.0,5.0,1.0,1.0,...,0.035,0.083,1.371429,0.130,2.714286,91.0,45.0,82.0,3.0,1.0
102,6.0,6,3.0,51.0,6.9,3.0,95.0,15.0,1.0,1.0,...,0.080,0.218,1.725000,0.302,2.775000,82.0,78.0,75.0,3.0,1.0


In [11]:
names = ['年龄', '大小', '临床分期', 'ER', 'PR', 'ER2', 'PR2', 'Her-2', '矫正后的KI67（2）', '分子分型2', 
              '病理类型', '病理分级', '基线最大（E0max）', '最大1', 'ΔEmaxt1', '最大2', 'ΔEmaxt2', '基线平均（E0mean）',
              '平均1', 'ΔEmeant1', '平均2', 'ΔEmeant2', '基线含血量（E0）', '含血量（E1）', 'Δ血t1', 
              '含血量（E2）', 'Δ血t2', '基线含氧量（E0）', '含氧量（E1）', 'Δ氧t1', '含氧量（E2）', 'Δ氧t2', 
              'HIF-1', 'TWIST-1', 'ITGB']

In [30]:
len(names)

35

In [12]:
results = ['RCB分级']
results2 = ['米勒分级']

In [13]:
df_x = df[names][0:80]

In [14]:
df_y = df[results][0:80]
df_y2 = df[results2][0:80]

In [15]:
X = np.array(df_x)
y = np.array(df_y)
y2 = np.array(df_y2)

In [16]:
y = y.ravel()
y2 = y2.ravel()

In [17]:
clf = svm.SVC(kernel='linear', decision_function_shape='ovo')
clf.fit(X, y)

clf2 = svm.SVC(kernel='linear', decision_function_shape='ovo')
clf2.fit(X, y2)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [18]:
X_test = np.array(df[names][81:104])
y_test = np.array(df[results][81:104]).ravel()
y_test2 = np.array(df[results2][81:104]).ravel()

In [19]:
y_predict = clf.predict(X_test)
y_predict2 = clf2.predict(X_test)

In [20]:
len(y_predict)

23

In [21]:
y_predict

array([3., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 3., 3., 3., 1., 1., 1.,
       2., 3., 2., 2., 3., 2.])

In [22]:
y_test

array([3., 1., 1., 1., 1., 1., 2., 2., 2., 2., 3., 3., 3., 3., 1., 2., 2.,
       2., 3., 3., 3., 3., 3.])

In [23]:
cm = np.array(confusion_matrix(y_test, y_predict, labels=[1,2,3]))
confusion = pd.DataFrame(cm, index=[1,2,3], columns=['predictive 1', 'predictive 2', 'predictive 3'])

cm2 = np.array(confusion_matrix(y_test2, y_predict2, labels=[1,2,3,4,5]))
confusion2 = pd.DataFrame(cm2, index=[1,2,3,4,5], columns=['predictive 1', 'predictive 2', 'predictive 3', 'predictive 4', 'predictive 5'])

In [24]:
cm

array([[6, 0, 0],
       [5, 2, 0],
       [0, 4, 6]], dtype=int64)

In [25]:
confusion

,predictive 1,predictive 2,predictive 3
1,6,0,0
2,5,2,0
3,0,4,6


In [26]:
confusion2

,predictive 1,predictive 2,predictive 3,predictive 4,predictive 5
1,4,1,1,0,0
2,1,1,4,0,0
3,0,1,3,0,1
4,0,0,2,1,0
5,0,0,1,0,2


In [27]:
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

         1.0       0.55      1.00      0.71         6
         2.0       0.33      0.29      0.31         7
         3.0       1.00      0.60      0.75        10

    accuracy                           0.61        23
   macro avg       0.63      0.63      0.59        23
weighted avg       0.68      0.61      0.60        23



In [28]:
print(classification_report(y_test2, y_predict2))

              precision    recall  f1-score   support

         1.0       0.80      0.67      0.73         6
         2.0       0.33      0.17      0.22         6
         3.0       0.27      0.60      0.37         5
         4.0       1.00      0.33      0.50         3
         5.0       0.67      0.67      0.67         3

    accuracy                           0.48        23
   macro avg       0.61      0.49      0.50        23
weighted avg       0.57      0.48      0.48        23



In [29]:
print('w = ',clf.coef_)
print('b = ',clf.intercept_)
print('Indices of support vectors = ', clf.support_)
print('Support vectors = ', clf.support_vectors_)
print('Number of support vectors for each class = ', clf.n_support_)
print('Coefficients of the support vector in the decision function = ', np.abs(clf.dual_coef_))

w =  [[ 1.33311034e-03  5.32810000e-01  1.31005876e-01  2.81835012e-02
   4.77320655e-02  4.33620812e-02  1.13283146e-02  7.98867096e-02
   5.46797354e-02 -6.18554584e-02  3.54245827e-02 -4.47996485e-02
   3.07141838e-02 -9.94720094e-02  6.94596006e-03  1.37253687e-01
   2.13966390e-02  1.91172584e-02  2.94367035e-01  2.31578130e-04
  -4.78368189e-01  6.95048503e-04 -7.48782469e-02 -1.14599610e-01
  -4.06892311e-02 -7.78301381e-02 -2.45402913e-02  3.56949281e-03
   8.73120103e-02  8.33393307e-02  9.15381735e-02 -3.74440675e-01
  -1.17371864e-01 -1.33202837e-01 -8.54191543e-03]
 [-5.48786172e-03  9.63049998e-04  0.00000000e+00 -1.25283198e-02
   1.38460219e-02  1.22144094e-05 -4.16127480e-04 -2.19017577e-04
   2.72692297e-02 -6.81414405e-04  0.00000000e+00  0.00000000e+00
   1.22125002e-02 -6.52612542e-04  2.91673070e-05 -1.66409331e-02
   1.08359963e-05 -8.56668335e-04 -4.63009433e-03 -2.15920365e-04
  -1.97959218e-02 -2.88252396e-04 -1.57298880e-04 -4.46238469e-04
  -2.05155885e-04 -5